In [1]:
! rm -rf gpHSP
!git clone https://github.com/aspuru-guzik-group/gpHSP

Cloning into 'gpHSP'...
remote: Enumerating objects: 104, done.
remote: Counting objects: 100% (66/66), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 104 (delta 37), reused 48 (delta 19), pack-reused 38
Receiving objects: 100% (104/104), 31.35 MiB | 31.16 MiB/s, done.
Resolving deltas: 100% (51/51), done.


# Assuming in a colab enviroment

In [2]:
import sys
if 'google.colab' in sys.modules:
    print('In colab!')
    sys.path.insert(0,'gpHSP')
    !pip install rdkit-pypi mordred ml_collections ngboost gpflow

In colab!


In [3]:
import os
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import scipy.stats as stats

import ngboost
import tensorflow as tf
import gpflow as gpf
import gphsp

gphsp.notebook_context()
gphsp.print_modules([ngboost, tf , gpf])

ngboost    = 0.3.12
tensorflow = 2.7.0
gpflow     = 2.3.1


## Utilities

In [4]:
at_data_dir = lambda x: os.path.join('gpHSP/data', x)
at_model_dir = lambda x: os.path.join('gpHSP/models', x)

# Load data and get all smiles

In [5]:
df = pd.read_csv(at_data_dir('Solvents_exp.csv'))
df['smiles'] = df['smiles'].apply(gphsp.get_isomeric_smiles)
gphsp.peek_df(df)

Index(['key', 'Type', 'δd', 'δp', 'δh', 'smiles', 'ID_type', 'ID', 'Ref',
       'organic', 'n_electrons', 'n_atoms', 'charge', 'MolWt', 'label',
       'finished', 'job_name', 'homo', 'lumo', 'gap', 'd-moments', 'dipole',
       'polar', 'run_time', 'n_cores', 'compute_time', 'Area', 'Hba', 'Hbd',
       'Volume', 'sigma_mom_0', 'sigma_mom_1', 'sigma_mom_2', 'sigma_mom_3',
       'sigma_mom_4', 'sigma_mom_5', 'sigma_norm', 'sigma_profile', 'drug'],
      dtype='object')
(193, 39)


,key,Type,δd,δp,δh,smiles,ID_type,ID,Ref,organic,n_electrons,n_atoms,charge,MolWt,label,finished,job_name,homo,lumo,gap,d-moments,dipole,polar,run_time,n_cores,compute_time,Area,Hba,Hbd,Volume,sigma_mom_0,sigma_mom_1,sigma_mom_2,sigma_mom_3,sigma_mom_4,sigma_mom_5,sigma_norm,sigma_profile,drug
0,"1,1,1-Trichloroethane",Solvent,16.8,4.3,2.0,CC(Cl)(Cl)Cl,CAS,71-55-6,1,True,32,5,0,133.405,Exp-0,True,SOLSPE_Exp-0,-7.097,-1.108,5.989,[-2.37335554 -0.11768285 0.07442233],2.377,64.374,94.214,4,376.856,1.326,0.0,0.0,0.126,0,16.6,-3.836,4.24,-2.021,1.54,41.428,[ 0. 0. 0. 0. ...,False


In [6]:
poly_df = pd.read_csv(at_data_dir('Polymers_exp.csv'))
poly_df['smiles'] = poly_df['smiles'].apply(gphsp.get_isomeric_smiles)
gphsp.peek_df(poly_df)

Index(['label', 'δd', 'δp', 'δh', 'smiles', 'test', 'poly_label', 'n_copies',
       'organic', 'n_electrons', 'n_atoms', 'charge', 'MolWt', 'n_frags',
       'largest', 'finished', 'job_name', 'poly_smiles', 'homo', 'lumo', 'gap',
       'd-moments', 'dipole', 'polar', 'run_time', 'n_cores', 'compute_time',
       'Area', 'Hba', 'Hbd', 'Volume', 'sigma_mom_0', 'sigma_mom_1',
       'sigma_mom_2', 'sigma_mom_3', 'sigma_mom_4', 'sigma_mom_5',
       'sigma_norm', 'sigma_profile'],
      dtype='object')
(31, 39)


,label,δd,δp,δh,smiles,test,poly_label,n_copies,organic,n_electrons,n_atoms,charge,MolWt,n_frags,largest,finished,job_name,poly_smiles,homo,lumo,gap,d-moments,dipole,polar,run_time,n_cores,compute_time,Area,Hba,Hbd,Volume,sigma_mom_0,sigma_mom_1,sigma_mom_2,sigma_mom_3,sigma_mom_4,sigma_mom_5,sigma_norm,sigma_profile
0,Polyacrylonitrile-n5,20.0,15.1,7.9,CCC#N,True,Polyacrylonitrile,5,True,110,45,0,275.4,1,True,True,SOLSPE2_Polyacrylonitrile-n5,[C@@H](C#N)(CCC#N)C[C@@H](C#N)C[C@H](C#N)CCC#N,-7.879,-1.12,6.759,[ 0.58302575 -8.49403297 -1.86749721],8.716,211.624,468.596,4,1874.384,3.047,1.539,0.22,0.343,0,175.0,26.1,154.0,51.4,165.0,66.77,[ 0. 0. 0. 0. ...


Load features

In [7]:
features = gphsp.SmilesMap(at_data_dir('mordred_features.npz'))

# Train rfHSP model on single molecules

In [9]:
save_model = True
model_suffix = 'HSP_ngboost_mol.pkl'

models = {}
x = features(df['smiles'].to_numpy(str))
names = gphsp.Y_COLS
print(names)
for name in tqdm(names, total=len(names)):
    y = df[name].to_numpy(np.float32).reshape(-1,1)
    model = ngboost.NGBRegressor().fit(x, y)
    y_dists = model.pred_dist(x)
    models[name] = model
    if save_model:
        fname = at_model_dir(f"{name}_{model_suffix}")
        gphsp.save_model(model, fname)

['δd', 'δp', 'δh']


  0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[iter 0] loss=2.0292 val_loss=0.0000 scale=1.0000 norm=1.5771
[iter 100] loss=1.3892 val_loss=0.0000 scale=2.0000 norm=1.7636
[iter 200] loss=0.7992 val_loss=0.0000 scale=2.0000 norm=1.2279
[iter 300] loss=0.3868 val_loss=0.0000 scale=1.0000 norm=0.4991
[iter 400] loss=0.0804 val_loss=0.0000 scale=1.0000 norm=0.4297


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[iter 0] loss=3.0726 val_loss=0.0000 scale=1.0000 norm=4.1787
[iter 100] loss=2.5094 val_loss=0.0000 scale=2.0000 norm=4.7659
[iter 200] loss=1.9639 val_loss=0.0000 scale=1.0000 norm=1.4583
[iter 300] loss=1.4908 val_loss=0.0000 scale=1.0000 norm=1.0831
[iter 400] loss=1.2050 val_loss=0.0000 scale=1.0000 norm=0.9079


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[iter 0] loss=3.1769 val_loss=0.0000 scale=1.0000 norm=4.6890
[iter 100] loss=2.3695 val_loss=0.0000 scale=2.0000 norm=3.6644
[iter 200] loss=1.6093 val_loss=0.0000 scale=2.0000 norm=2.0204
[iter 300] loss=0.9949 val_loss=0.0000 scale=2.0000 norm=1.5120
[iter 400] loss=0.6177 val_loss=0.0000 scale=1.0000 norm=0.6421


# Train gpHSP model on polymers

In [11]:
model_suffix = 'HSP_ngboost_mol.pkl'
names = gphsp.Y_COLS
models = {name: gphsp.load_model(at_model_dir(f"{name}_{model_suffix}")) for name in names}
mol_x = features(poly_df['smiles'].to_numpy(str))
x = gphsp.predictions_as_features(mol_x, models)

(31, 6)


In [21]:
save_models = True
poly_models = {}
parts = {'mean':[0,1, 2], 'std':[3, 4, 5]}
for name in tqdm(names, total=len(names)):
    y = poly_df[name].to_numpy(np.float64).reshape(-1, 1)
    model = gphsp.make_gp(x,y, parts, use_ard=True)
    poly_models[name] = model
    gpf.utilities.print_summary(model)
    if save_models:
        adir = at_model_dir(f'{name}_gpHSP_polymer')
        gphsp.save_gp(model, adir)

  0%|          | 0/3 [00:00<?, ?it/s]

name,class,transform,prior,trainable,shape,dtype,value
GPR.mean_function.c,Parameter,Identity,,True,"(1,)",float64,[17.97759602]
GPR.kernel.kernels[0].variance,Parameter,Softplus,,True,(),float64,0.29748370193997825
GPR.kernel.kernels[0].lengthscales,Parameter,Softplus,,True,(),float64,8.833983162675848
GPR.kernel.kernels[1].variance,Parameter,Softplus,,True,(),float64,0.29748370193997825
GPR.kernel.kernels[1].lengthscales,Parameter,Softplus,,True,(),float64,8.833983162675848
GPR.likelihood.variance,Parameter,Softplus + Shift,,True,(),float64,0.4386974356512946


dims_i = <class 'numpy.ndarray'>
Instructions for updating:
`min_event_ndims` is now static for all bijectors; this property is no longer needed.


Instructions for updating:
`min_event_ndims` is now static for all bijectors; this property is no longer needed.


INFO:tensorflow:Assets written to: gpHSP/models/δd_gpHSP_polymer/assets


...

dims_i = <class 'numpy.ndarray'>
INFO:tensorflow:Assets written to: gpHSP/models/δp_gpHSP_polymer/assets


INFO:tensorflow:Assets written to: gpHSP/models/δp_gpHSP_polymer/assets


name,class,transform,prior,trainable,shape,dtype,value
GPR.mean_function.c,Parameter,Identity,,True,"(1,)",float64,[7.57579405]
GPR.kernel.kernels[0].variance,Parameter,Softplus,,True,(),float64,6.400869690324182
GPR.kernel.kernels[0].lengthscales,Parameter,Softplus,,True,(),float64,2.6753462983994334
GPR.kernel.kernels[1].variance,Parameter,Softplus,,True,(),float64,6.400869690324182
GPR.kernel.kernels[1].lengthscales,Parameter,Softplus,,True,(),float64,2.6753462983994334
GPR.likelihood.variance,Parameter,Softplus + Shift,,True,(),float64,6.892437309826844


dims_i = <class 'numpy.ndarray'>
INFO:tensorflow:Assets written to: gpHSP/models/δh_gpHSP_polymer/assets


INFO:tensorflow:Assets written to: gpHSP/models/δh_gpHSP_polymer/assets
